In [ ]:
# Define model
def DefineModel3(Model):
    Model.Racks = pyo.Var(domain = pyo.NonNegativeIntegers, initialize = 0)   # Number of racks
    Model.Allocation = pyo.Var(Model.P, Model.S, domain = pyo.Binary, initialize = 0)   # Allocation of pallets to shelves and racks

    def rule_fit(Model, P):   # Each pallet must be allocated to a shelf that is at least the height of the pallet
        return sum(Model.ShelfHeights[s] * Model.Allocation[P, s] for s in Model.S) >= Model.Pallets[P]
    Model.PalletFits = pyo.Constraint(Model.P, rule = rule_fit)

    def rule_use(Model, P):   # Each pallet must be allocated to exactly one shelf
        return sum(Model.Allocation[P, s] for s in Model.S) == 1
    Model.MustUse = pyo.Constraint(Model.P, rule = rule_use)

    def rule_within(Model, S):   # Times each shelf size is allocated to a pallet must be no larger than the number of racks
        return sum(Model.Allocation[p, S] for p in Model.P) <= Model.Racks * Model.PalletsPerShelf   # Some shelves may be empty (Allocation = 0)
    Model.WithinRack = pyo.Constraint(Model.S, rule = rule_within)
    
    def rule_Obj(Model):   # Minimize the number of racks we need to allocate all pallets to a shelf
        if WeightedObj:   # Weighted to also minimize number of shelves in a rack, if required
            return Model.WeightRacks * Model.Racks + Model.WeightShelves * Model.NumShelves
        else:
            return Model.Racks
    Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.minimize)